# Frequency

In [7]:
%config IPCompleter.greedy=True

In [8]:
import pandas as pd

feature_pos = [[0,8],[8,12],[12,28],[28,44],[44,52],[52,64],[64,80],[80,92],[92,102]] 
df_train = pd.read_csv('train_kaggle.csv')
df_test = pd.read_csv('sample_solution.csv')
Y = df_train['Label'].values

## Save Frequency Features
Skip this if you have saved the features

In [ ]:
import numpy as np

dataframes = np.load('allData.npy', allow_pickle = True)
dataframes.shape

def load_test_dataframe(id):
    test_data = np.load("test/test/{}.npy".format(id))
    return pd.DataFrame(data=test_data)

testdatas = []
for id in df_test['Id']:
    dfi = load_test_dataframe(id)
    testdatas.append(dfi.values)

In [ ]:
feat = dataframes[0].shape[1]
# feat = 12

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm

def one_hot_encoding(encoder, dfs, col):
    if encoder == None:
        data = set(dfs[0][:,col])
        for i in range(1,len(dfs)):
            df = dfs[i]
            data |= set(df[:,col])
        encoder = OneHotEncoder(handle_unknown='ignore')
        encoder = encoder.fit([[str(e)] for e in data])
    encodedData = []
    for df in dfs:
        mat = np.sum(encoder.transform([[str(e)] for e in df[:,col]]), axis=0)
        mat = np.squeeze(np.asarray(mat))
        encodedData.append(mat)
    encodedData = np.stack(encodedData)    
    return encoder, encodedData

def one_hot_encoding_feature(encoders, dfs, feature):
    if len(encoders) == 0:
        encoder, data = one_hot_encoding(None, dfs, feature_pos[feature][0])
        encoders.append(encoder)    
        for col in tqdm(range(feature_pos[feature][0] + 1,feature_pos[feature][1])):
            encoder, dataCol = one_hot_encoding(None, dfs, col)
            encoders.append(encoder)
            data = np.concatenate((data, dataCol), axis=1)
    else:
        encoder, data = one_hot_encoding(encoders[0], testdatas, feature_pos[feature][0])
        for col in tqdm(range(feature_pos[feature][0] + 1,feature_pos[feature][1])):
            i = col - feature_pos[feature][0]
            encoder, dataCol = one_hot_encoding(encoders[i], testdatas, col)
            data = np.concatenate((data, dataCol), axis=1)
    return encoders, data

In [ ]:
import gc

# Give up last feature
for feat in range(len(feature_pos) - 1):
    print(feat)
    encoders, train = one_hot_encoding_feature([], dataframes, feat)
    encoders, test = one_hot_encoding_feature(encoders, testdatas, feat)
    np.save('freq/' + str(feat) + '_train.npy', train)
    np.save('freq/' + str(feat) + '_test.npy', test)
    gc.collect()

## Load Frequency Features

In [9]:
import numpy as np
# last feature must be false
feature_selected = [True,True,True,True,True,True,True,True,False] 

def load_freq(feat, postfix):
    data = np.load("freq/{}_{}.npy".format(feat, postfix))
    return data

def load_test_freq(feat):
    return load_freq(feat, 'test')

def load_train_freq(feat):
    return load_freq(feat, 'train')

In [10]:
def select_features(feature_selected):
    XTrain = []
    XTest = []
    for feat in range(len(feature_pos)):
        if feature_selected[feat] == False:
            continue
        train = load_train_freq(feat)
        test = load_test_freq(feat)
        XTrain.append(train)
        XTest.append(test)
    XTrain = np.concatenate(XTrain, axis=1)
    XTest = np.concatenate(XTest, axis=1)
    return XTrain, XTest

def all_not_selected(feature_selected):
    for i in range(len(feature_pos)):
        if feature_selected[i] == True:
            return False
    return True

In [12]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn import metrics
import gc

scores = [0]
f0s = [0]
f1s = [0]
f2s = [0]
f3s = [0]
f4s = [0]
f5s = [0]
f6s = [0]
f7s = [0]
score = (0, 0, 0, 0, 0, 0, 0, 0, 0)
cnt = 0
for f0 in range(2):
    feature_selected[0] = bool(f0)
    for f1 in range(2):
        feature_selected[1] = bool(f1)
        for f2 in range(2):
            feature_selected[2] = bool(f2)
            for f3 in range(2):
                feature_selected[3] = bool(f3)
                for f4 in range(2):
                    feature_selected[4] = bool(f4)
                    for f5 in range(2):
                        feature_selected[5] = bool(f5)
                        for f6 in range(2):
                            feature_selected[6] = bool(f6)
                            for f7 in range(2):
                                cnt += 1
                                feature_selected[7] = bool(f7)
                                if all_not_selected(feature_selected) == True:
                                    continue
                                if cnt % 10 == 0:
                                    print(cnt)
                                XTrain, XTest = select_features(feature_selected)
                                del XTest
                                XTrain, X_valid, y_train, y_valid = train_test_split(XTrain, Y, test_size=0.2, random_state=42)
                                gc.collect()
                                model = LGBMClassifier()
                                model.fit(XTrain, y_train)
                                pred = model.predict_proba(X_valid)[:,1]
                                fpr, tpr, thresholds = metrics.roc_curve(y_valid, pred)
                                auc_score = metrics.auc(fpr, tpr)
                                scores.append(auc_score)
                                f0s.append(f0)
                                f1s.append(f1)
                                f2s.append(f2)
                                f3s.append(f3)
                                f4s.append(f4)
                                f5s.append(f5)
                                f6s.append(f6)
                                f7s.append(f7)
                                gc.collect()
                                if score[0] < auc_score:
                                    score = (auc_score, f0, f1, f2, f3, f4, f5, f6, f7)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250


In [17]:
auc_csv = pd.DataFrame({'f0': f0s, 'f1': f1s, 'f2': f2s, 'f3': f3s, 'f4': f4s, 'f5': f5s, 'f6': f6s, 'f7': f7s, 'auc': scores}).sort_values(by=['auc'])
auc_csv.to_csv('freq_auc.csv')
auc_csv

,f0,f1,f2,f3,f4,f5,f6,f7,auc
0,0,0,0,0,0,0,0,0,0.000000
2,0,0,0,0,0,0,1,0,0.882728
1,0,0,0,0,0,0,0,1,0.889711
3,0,0,0,0,0,0,1,1,0.895471
5,0,0,0,0,0,1,0,1,0.980764
4,0,0,0,0,0,1,0,0,0.980828
6,0,0,0,0,0,1,1,0,0.981217
7,0,0,0,0,0,1,1,1,0.982000
11,0,0,0,0,1,0,1,1,0.982529
18,0,0,0,1,0,0,1,0,0.982658


In [15]:
print(score)

(0.992238129804707, 1, 1, 1, 1, 0, 0, 0, 0)


In [16]:
for i in range(8):
    feature_selected[i] = bool(score[i + 1])

XTrain, XTest = select_features(feature_selected)

## Try Others

In [28]:
feature_selected = [True,True,True,False,True,False,True,False,False] 
XTrain, XTest = select_features(feature_selected)

## Train models

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(XTrain, Y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(14929, 69209)
(3733, 69209)
(14929,)
(3733,)


In [31]:
import lightgbm as lgb

train_set = lgb.Dataset(X_train.astype(np.int32), y_train.astype(int))
valid_set = lgb.Dataset(X_valid.astype(np.int32), y_valid.astype(int))

from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(XTrain, Y)
modelSelection = SelectFromModel(clf, prefit=True, max_features=8000)
XTrain = modelSelection.transform(XTrain)
XTrain.shape

XTest = modelSelection.transform(XTest)
XTest.shape

In [32]:
from hyperopt import hp
from hyperopt import Trials
from hyperopt import fmin
from hyperopt import tpe
import lightgbm as lgb
from hyperopt import STATUS_OK

ITER = 50
STOP_ROUND = 5

# Create the dataset
def objective(params):
    """Objective function for Gradient Boosting Machine Hyperparameter Tuning"""
    params['num_leaves'] = int(params['num_leaves'])
    params['subsample_for_bin'] = int(params['subsample_for_bin'])
    params['min_child_samples'] = int(params['min_child_samples'])
    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evalute based on ROC AUC
    bst = lgb.train(params, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)
    bst.save_model('model.txt', num_iteration=bst.best_iteration)
  
    # Extract the best score
    best_score = bst.best_score['valid_0']['auc']
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

# Define the search space
space = {
    'boosting_type': hp.choice('boosting_type', 
                                [
                                    'gbdt',
                                    'dart',
                                    'goss'
                                ]),
    'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    'metric': 'auc'
}

# Trials object to track progress
bayes_trials = Trials()

MAX_EVALS = 500

# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials)
best

[1]	valid_0's auc: 0.952329                                                                                 
Training until validation scores don't improve for 5 rounds                                                 
[2]	valid_0's auc: 0.957452                                                                                 
[3]	valid_0's auc: 0.96146                                                                                  
[4]	valid_0's auc: 0.964688                                                                                 
[5]	valid_0's auc: 0.965831                                                                                 
[6]	valid_0's auc: 0.965823                                                                                 
[7]	valid_0's auc: 0.965304                                                                                 
[8]	valid_0's auc: 0.96567                                                                                  
[9]	valid_0's auc: 

[9]	valid_0's auc: 0.733453                                                                                 
[10]	valid_0's auc: 0.741382                                                                                
[11]	valid_0's auc: 0.732563                                                                                
Early stopping, best iteration is:                                                                          
[6]	valid_0's auc: 0.96985
[1]	valid_0's auc: 0.959583                                                                                 
Training until validation scores don't improve for 5 rounds                                                 
[2]	valid_0's auc: 0.962912                                                                                 
[3]	valid_0's auc: 0.962915                                                                                 
[4]	valid_0's auc: 0.965686                                                                          

[18]	valid_0's auc: 0.979395                                                                                
[19]	valid_0's auc: 0.979544                                                                                
[20]	valid_0's auc: 0.979623                                                                                
[21]	valid_0's auc: 0.979823                                                                                
[22]	valid_0's auc: 0.980028                                                                                
[23]	valid_0's auc: 0.980327                                                                                
[24]	valid_0's auc: 0.980415                                                                                
[25]	valid_0's auc: 0.98052                                                                                 
[26]	valid_0's auc: 0.980508                                                                                
[27]	valid_0's auc:

C:\Users\jw.chan\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\callback.py:192: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')



[2]	valid_0's auc: 0.965754                                                                                 
[3]	valid_0's auc: 0.967219                                                                                 
[4]	valid_0's auc: 0.96947                                                                                  
[5]	valid_0's auc: 0.969428                                                                                 
[6]	valid_0's auc: 0.970318                                                                                 
[7]	valid_0's auc: 0.97127                                                                                  
[8]	valid_0's auc: 0.971537                                                                                 
[9]	valid_0's auc: 0.971679                                                                                 
[10]	valid_0's auc: 0.971862                                                                                
[11]	valid_0's auc:

...

[25]	valid_0's auc: 0.988587                                                                                
[26]	valid_0's auc: 0.988892                                                                                
[27]	valid_0's auc: 0.989025                                                                                
[28]	valid_0's auc: 0.989227                                                                                
[29]	valid_0's auc: 0.98937                                                                                 
[30]	valid_0's auc: 0.989481                                                                                
[31]	valid_0's auc: 0.989574                                                                                
[32]	valid_0's auc: 0.989532                                                                                
[33]	valid_0's auc: 0.989603                                                                                
[34]	valid_0's auc:

[5]	valid_0's auc: 0.9809                                                                                   
[6]	valid_0's auc: 0.981171                                                                                 
[7]	valid_0's auc: 0.98169                                                                                  
[8]	valid_0's auc: 0.982083                                                                                 
[9]	valid_0's auc: 0.982106                                                                                 
[10]	valid_0's auc: 0.982248                                                                                
[11]	valid_0's auc: 0.982454                                                                                
[12]	valid_0's auc: 0.982733                                                                                
[13]	valid_0's auc: 0.9828                                                                                  
[14]	valid_0's auc:

[27]	valid_0's auc: 0.986518                                                                                
[28]	valid_0's auc: 0.986629                                                                                
[29]	valid_0's auc: 0.986801                                                                                
[30]	valid_0's auc: 0.986931                                                                                
[31]	valid_0's auc: 0.987026                                                                                
[32]	valid_0's auc: 0.98719                                                                                 
[33]	valid_0's auc: 0.987279                                                                                
[34]	valid_0's auc: 0.987424                                                                                
[35]	valid_0's auc: 0.987456                                                                                
[36]	valid_0's auc:

Early stopping, best iteration is:                                                                          
[43]	valid_0's auc: 0.990287
100%|████████████████████████████████████| 500/500 [11:39<00:00,  1.40s/it, best loss: 0.007642490445247119]


{'boosting_type': 0,
 'colsample_by_tree': 0.827439321864023,
 'learning_rate': 0.17269696108520272,
 'min_child_samples': 20.0,
 'num_leaves': 102.0,
 'reg_alpha': 0.09087255405542816,
 'reg_lambda': 0.6621447542749292,
 'subsample_for_bin': 280000.0}

In [33]:
best['boosting_type'] = 'dart'
best['num_leaves'] = int(best['num_leaves'])
best['subsample_for_bin'] = int(best['subsample_for_bin'])
best['min_child_samples'] = int(best['min_child_samples'])
lgbModel = lgb.train(best, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)
lgbModel.save_model('model.txt', num_iteration=lgbModel.best_iteration)
YTest = lgbModel.predict(XTest)

In [34]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(boosting_type=best['boosting_type'],
                        num_leaves=best['num_leaves'],
                        learning_rate=best['learning_rate'],
                        subsample_for_bin=best['subsample_for_bin'],
                        min_child_samples=best['min_child_samples'],
                        reg_alpha=best['reg_alpha'],
                        reg_lambda=best['reg_lambda'],
                        colsample_bytree=best['colsample_by_tree'])
model.fit(XTrain, Y)
probs = model.predict_proba(XTest, num_iteration=lgbModel.best_iteration)

In [35]:
YTest = probs[:, 0]

In [36]:
df_test['Predicted'] = YTest
df_test

,Id,Predicted
0,0,0.000991
1,1,0.005953
2,2,0.998827
3,3,0.018138
4,4,0.467886
5,5,0.912315
6,6,0.999092
7,7,0.924186
8,8,0.000991
9,9,0.001162


In [37]:
df_test.to_csv('test.csv', index=False)